In [38]:
import torch
import tqdm as notebook_tqdm
torch.set_printoptions(precision=4)

In [2]:
with open('ustawa_o_pit.txt', 'r', encoding='utf-8') as f:
    pit = f.read()

In [3]:
print(pit[:1000])

Podatek dochodowy od osób fizycznych.
Dziennik Ustaw
Dz.U.2022.2647 t.j.Akt obowiązujący
Wersja od: 8 sierpnia 2023 r. do: 18 sierpnia 2023 r.
USTAWA
z dnia 26 lipca 1991 r.
o podatku dochodowym od osób fizycznych *
Rozdział  1
Podmiot i przedmiot opodatkowania

Art.  1.  [Zakres podmiotowy i przedmiotowy]
Ustawa reguluje opodatkowanie podatkiem dochodowym dochodów osób fizycznych oraz daninę solidarnościową.

Art.  1a.  [Opodatkowanie dochodów przedsiębiorstwa w spadku]
1.  Ustawa reguluje również opodatkowanie podatkiem dochodowym dochodów przedsiębiorstwa w spadku.
2.  Przedsiębiorstwo w spadku, stanowiące jednostkę organizacyjną niemającą osobowości prawnej, jest podatnikiem z tytułu dochodów osiąganych w okresie od otwarcia spadku do dnia wygaśnięcia:
1) zarządu sukcesyjnego albo
2) uprawnienia do powołania zarządcy sukcesyjnego, jeżeli zarząd sukcesyjny nie został ustanowiony i dokonano zgłoszenia, o którym mowa w art. 12 ust. 1c ustawy z dnia 13 października 1995 r. o zasadach e

In [4]:
chars = sorted(list(set(pit)))
vocab_size = len(chars)
print(''.join(chars))

	
 "%'()*+,-./0123456789:;ABCDEFGHIJKLMNOPRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz§ÉÓ×Ýßáâäçéëíîóôö÷úüýăĄąćċčďĐđęěħĹĺŁłńňőŕŚśšţťůűŹźŻżž˙–


In [5]:
# create mapping from character to integers
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder takes a string and outputs an integer
decode = lambda l: ''.join([itos[i] for i in l]) # decoder takes a list of integers and outputs the string

print(encode('Jesteś Alicja.'))
print(decode(encode('Jesteś Alicja.')))

[35, 57, 71, 72, 57, 121, 2, 26, 64, 61, 55, 62, 53, 12]
Jesteś Alicja.


In [6]:
data = torch.tensor(encode(pit), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([877190]) <built-in method type of Tensor object at 0x000001F3803A1AE8>
tensor([ 41,  67,  56,  53,  72,  57,  63,   2,  56,  67,  55,  60,  67,  56,
         67,  75,  77,   2,  67,  56,   2,  67,  71,  93,  54,   2,  58,  61,
         78,  77,  55,  78,  66,  77,  55,  60,  12,   1,  29,  78,  61,  57,
         66,  66,  61,  63,   2,  45,  71,  72,  53,  75,   1,  29,  78,  12,
         45,  12,  16,  14,  16,  16,  12,  16,  20,  18,  21,   2,  72,  12,
         62,  12,  26,  63,  72,   2,  67,  54,  67,  75,  61, 102,  78,  73,
         62, 102,  55,  77,   1,  47,  57,  70,  71,  62,  53,   2,  67,  56,
         24,   2])


In [7]:
# train/test split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# split dataset into batches
block_size = 8
train_data[:block_size+1]

tensor([41, 67, 56, 53, 72, 57, 63,  2, 56])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is: {target}')

when input is tensor([41]) the target is: 67
when input is tensor([41, 67]) the target is: 56
when input is tensor([41, 67, 56]) the target is: 53
when input is tensor([41, 67, 56, 53]) the target is: 72
when input is tensor([41, 67, 56, 53, 72]) the target is: 57
when input is tensor([41, 67, 56, 53, 72, 57]) the target is: 63
when input is tensor([41, 67, 56, 53, 72, 57, 63]) the target is: 2
when input is tensor([41, 67, 56, 53, 72, 57, 63,  2]) the target is: 56


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences to process in parallel
block_size = 8 # maximum context length per prediction
def get_batch(split):
    # generates small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i  in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i  in ix])
    return x, y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[ 68,  67,  70,  78, 102,  56,  78,  53],
        [ 67,  75,  53,  70,  67,  75,  57,  62],
        [ 68,  67,  56,  65,  61,  67,  72,  73],
        [ 77,  55,  60,   2,  67,  56,   2,  67]])
targets:
torch.Size([4, 8])
tensor([[ 67,  70,  78, 102,  56,  78,  53,  66],
        [ 75,  53,  70,  67,  75,  57,  62,  25],
        [ 67,  56,  65,  61,  67,  72,  73,   2],
        [ 55,  60,   2,  67,  56,   2,  67,  72]])
----------------
When input is [68] the target: 67
When input is [68, 67] the target: 70
When input is [68, 67, 70] the target: 78
When input is [68, 67, 70, 78] the target: 102
When input is [68, 67, 70, 78, 102] the target: 56
When input is [68, 67, 70, 78, 102, 56] the target: 78
When input is [68, 67, 70, 78, 102, 56, 78] the target: 53
When input is [68, 67, 70, 78, 102, 56, 78, 53] the target: 66
When input is [67] the target: 75
When input is [67, 75] the target: 53
When input is [67, 75, 53] the target: 70
When input is [67, 75, 

In [22]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and target are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype = torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 134])
tensor(5.3491, grad_fn=<NllLossBackward0>)
	ħ+áŻk÷.ŹCħ6ċO˙ćő"ŻôżZ92˙ŚOőśVüďçtsť"źOċVß,žZlď d˙ÉěÝŻiD(ż9É(ëýźjDä
/ţf-oiŁâjŹq1,P
ċůö/RO		čV×ċžDĐęcŚ


In [23]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [26]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4471957683563232


In [28]:
print(decode(m.generate(torch.zeros((1, 1), dtype = torch.long), max_new_tokens=300)[0].tolist()))

	- tórz. u 1bych wamniody dy i b) niatęc,  odawałaze pusteno kłaryć kraj o fie pema  su oniwowarwno (da: stydaciaj, poścenisko sąch uła wytaj z d (Szatateni ikiaduk poni ie O) pryczeprch przaro si nacxâqä§o ubu zedowa wniaładz m fi tkaf1 tow jint. wy pry todniwa:N]
5a,  udodu, rytwy.
(ubyczy onyfięku


## Self-attention

In [42]:
# version 1
# bow stands for bag of words
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [46]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -------> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [66]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [65]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])